# Day 18: Many-Worlds Interpretation

## Part 1

As you approach Neptune, a planetary security system detects you and activates 
a giant tractor beam on Triton! You have no choice but to land.

A scan of the local area reveals only one interesting feature: a massive 
underground vault. You generate a map of the tunnels (your puzzle input). The 
tunnels are too narrow to move diagonally.

Only one **entrance** (marked `@`) is present among the **open passages** 
(marked `.`) and stone walls (#), but you also detect an assortment of 
**keys** (shown as lowercase letters) and **doors** (shown as uppercase 
letters). Keys of a given letter open the door of the same letter: `a` opens 
`A`, `b` opens `B`, and so on. You aren't sure which key you need to disable 
the tractor beam, so you'll need to **collect all of them**.

For example, suppose you have the following map:

```
#########
#b.A.@.a#
#########
```

Starting from the entrance (`@`), you can only access a large door (`A`) and 
a key (`a`). Moving toward the door doesn't help you, but you can move `2` steps 
to collect the key, unlocking `A` in the process:

```
#########
#b.....@#
#########
```

Then, you can move `6` steps to collect the only other key, `b`:

```
#########
#@......#
#########
```

So, collecting every key took a total of **`8`** steps.

Here is a larger example:

```
########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################
```

The only reasonable move is to take key `a` and unlock door `A`:

```
########################
#f.D.E.e.C.b.....@.B.c.#
######################.#
#d.....................#
########################
```

Then, do the same with key `b`:

```
########################
#f.D.E.e.C.@.........c.#
######################.#
#d.....................#
########################
```

...and the same with key `c`:

```
########################
#f.D.E.e.............@.#
######################.#
#d.....................#
########################
```

Now, you have a choice between keys `d` and `e`. While key `e` is closer, 
collecting it now would be slower in the long run than collecting key `d` 
first, so that's the best choice:

```
########################
#f...E.e...............#
######################.#
#@.....................#
########################
```

Finally, collect key `e` to unlock door `E`, then collect key `f`, taking a 
grand total of **`86`** steps.

Here are a few more examples:

- 
```
########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################
```

Shortest path is `132` steps: `b, a, c, d, f, e, g`

- 
```
#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################
```

Shortest paths are `136` steps;
one is: `a, f, b, j, g, n, h, d, l, o, e, p, c, i, k, m`

- 
```
########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################
```

Shortest paths are `81` steps; one is: `a, c, f, i, d, g, b, e, h`

**How many steps is the shortest path that collects all of the keys?**

In [2]:
from pathlib import Path

import numpy as np
import itertools
import networkx as nx
import string
from collections import defaultdict

input_path = Path("..") / "inputs" / "day_18.txt"
inputs = input_path.read_text()
inputs = inputs.split("\n")
inputs = [list(line) for line in inputs]
input_arr = np.array(inputs)

### Test Cases

In [185]:
test_cases = [
    (["#########","#b.A.@.a#","#########"], 8),
    ([
        "########################",
        "#f.D.E.e.C.b.A.@.a.B.c.#",
        "######################.#",
        "#d.....................#",
        "########################"
    ], 86),
    ([
        "########################",
        "#...............b.C.D.f#",
        "#.######################",
        "#.....@.a.B.c.d.A.e.F.g#",
        "########################"
    ], 132),
    ([
        "#################",
        "#i.G..c...e..H.p#",
        "########.########",
        "#j.A..b...f..D.o#",
        "########@########",
        "#k.E..a...g..B.n#",
        "########.########",
        "#l.F..d...h..C.m#",
        "#################"
    ], 136),
    ([
        "########################",
        "#@..............ac.GI.b#",
        "###d#e#f################",
        "###A#B#C################",
        "###g#h#i################",
        "########################"
    ], 81)
    
]

In [233]:
maze = ["#########","#b.A.@.a#","#########"]
maze = [
        "#################",
        "#i.G..c...e..H.p#",
        "########.########",
        "#j.A..b...f..D.o#",
        "########@########",
        "#k.E..a...g..B.n#",
        "########.########",
        "#l.F..d...h..C.m#",
        "#################"
    ]

In [234]:
maze_dict = {}
for y, line in enumerate(maze):
    for x, val in enumerate(line):
        maze_dict[(x, y)] = val

In [235]:


def find_shortest_path(start, end, maze_dict):
#     print(f"looking for shortest path between {maze_dict[start]} - {maze_dict[end]}")
    num_keys = sum(i in string.ascii_lowercase for i in maze_dict.values())

    lowercase_dict = {a:b for b, a in enumerate(string.ascii_lowercase)}
    uppercase_dict = {a:b for b, a in enumerate(string.ascii_uppercase)}
    hashes = {k for k,v in maze_dict.items() if v == "#"}
    key_grid = {k for k, v in maze_dict.items() if v in string.ascii_lowercase}
    
    keys = "0"*num_keys
    if maze_dict[start] in string.ascii_lowercase:
        
        char_index = lowercase_dict[maze_dict[start]]
        keys = keys[:char_index]+"1"+keys[char_index+1:]

    queue = [[start, 0, keys]]
    v = defaultdict(dict)

    new_keybits = ["0"]    
    while len(queue):
        location, length, keybits = queue.pop(0)
#         print(f"at {location} with keys {keybits}")
        if location in v.get(keybits, {}):
            continue
        else:
            v[keybits][location] = length + 1
        next_bits = (
            (location[0]+1, location[1]),
            (location[0]-1, location[1]),
            (location[0], location[1]+1),
            (location[0], location[1]-1)
        )
        next_bits = (i for i in next_bits if i not in hashes)
        next_bits = (i for i in next_bits if i not in v[keybits])

        for those_coords in next_bits:
            if those_coords == end:
#                 print(f"found shortest path of {length + 1}, {keybits}")
                return length + 1

            t = [those_coords, length+1, keybits]
            val_there = maze_dict[those_coords]

            if val_there in string.ascii_uppercase:
#                 print(f"hit gate {val_there}")
                char_index = uppercase_dict[val_there]
                if keybits[char_index] == "0":
                    continue
            elif those_coords in key_grid and those_coords != start:
                
                char_index = lowercase_dict[val_there]
                existing = SHORTEST_PATHS.get((maze_dict[start], val_there), float("inf"))
                if  existing > length+1: 
#                     print(f"existing shortest path for {last_key}, {those_coords}")
#                     print(f"actually found shortest path between {maze_dict[start]} and {val_there}, {length+1}")
                    if (maze_dict[start], val_there) not in SHORTEST_PATHS:
                        SHORTEST_PATHS[(maze_dict[start], val_there)] = length + 1
                    if (val_there, maze_dict[end]) in SHORTEST_PATHS:
#                         print(f"and we know the distance between {val_there} and {maze_dict[end]}")
                        return length + 1 + SHORTEST_PATHS[(val_there, maze_dict[end])]
                new_keybits = keybits[:char_index]+"1"+keybits[char_index+1:]
#                 new_keybits = ("0"*char_index)+"1"+("0"* (len(keybits) - char_index))
#                 print(f"now looking for shortest_path between {those_coords} and {end}, {maze_dict[those_coords]} - {maze_dict[end]}")
                t[2] = new_keybits
#                 t[1] = 0

            queue.append(t)
#     print(f"no path found between {maze_dict[start]} - {maze_dict[end]}")
    return None

In [236]:
start = [k for k, v in maze_dict.items() if v == "@"][0]
keys= [(k, v) for k, v in maze_dict.items() if v in string.ascii_lowercase]
SHORTEST_PATHS = {}

In [237]:
for (location_1, key_1), (location_2, key_2) in itertools.permutations(keys, 2):
    if key_1 == key_2:
        continue
    if (key_1, key_2) in SHORTEST_PATHS:
        continue
    length = find_shortest_path(location_1, location_2, maze_dict)
#     print(location_1, location_2)
#     print(s, key_2, length)
    if (key_1, key_2) not in SHORTEST_PATHS :
#         print(f"setting {key_1}, {key_2} to {length}")
        if length:
            SHORTEST_PATHS[(key_1, key_2)] = length

In [238]:
for location, key in keys:
    length = find_shortest_path(start, location, maze_dict)
    if length:
        SHORTEST_PATHS[("@", key)] = length

In [239]:
rev_keys = {v:k for k, v in keys}
rev_keys["@"] = start

In [240]:
find_shortest_path(rev_keys["@"], rev_keys["a"], maze_dict)

3

In [241]:
find_shortest_path(rev_keys["a"], rev_keys["f"], maze_dict)

6

In [242]:
find_shortest_path(rev_keys["f"], rev_keys["b"], maze_dict)

4

In [243]:
find_shortest_path(rev_keys["b"], rev_keys["j"], maze_dict)

17

In [245]:
find_shortest_path(rev_keys["a"], rev_keys["g"], maze_dict)

4

In [209]:
SHORTEST_PATHS[("f", "e")]

6

In [210]:
SHORTEST_PATHS[(("e", "g"))]

8

In [200]:
G = nx.DiGraph()

In [201]:
G.add_node("@")
for location, key in keys:
    G.add_node(key)

In [202]:
for (start, end), weight in SHORTEST_PATHS.items():
    if weight:
        G.add_edge(start, end, weight=weight)

In [192]:

path = nx.algorithms.shortest_paths.weighted.single_source_dijkstra(G, "@", weight="weight")
print(path)
#     if len(path) == len(keys) + 1:
#         print(nx.algorithms.shortest_paths.weighted.dijkstra_path_length(G, "@", key))

({'@': 0, 'g': 3, 'a': 3, 'f': 3, 'b': 3, 'h': 5, 'd': 5, 'e': 9, 'c': 9, 'j': 14, 'n': 14, 'k': 18, 'i': 20, 'l': 20, 'o': 22, 'm': 24, 'p': 24}, {'@': ['@'], 'g': ['@', 'g'], 'i': ['@', 'a', 'i'], 'a': ['@', 'a'], 'c': ['@', 'b', 'c'], 'f': ['@', 'f'], 'e': ['@', 'e'], 'b': ['@', 'b'], 'p': ['@', 'a', 'p'], 'h': ['@', 'h'], 'j': ['@', 'a', 'j'], 'd': ['@', 'd'], 'o': ['@', 'a', 'o'], 'k': ['@', 'k'], 'n': ['@', 'b', 'n'], 'l': ['@', 'b', 'l'], 'm': ['@', 'm']})


In [193]:
SHORTEST_PATHS

{('a', 'g'): 4,
 ('a', 'h'): 6,
 ('a', 'd'): 6,
 ('a', 'f'): 6,
 ('a', 'b'): 6,
 ('a', 'e'): 8,
 ('a', 'c'): 8,
 ('c', 'e'): 4,
 ('c', 'f'): 6,
 ('c', 'b'): 6,
 ('c', 'g'): 8,
 ('c', 'a'): 8,
 ('c', 'h'): 10,
 ('c', 'd'): 10,
 ('c', 'm'): 15,
 ('c', 'k'): 17,
 ('c', 'n'): 17,
 ('c', 'l'): 19,
 ('c', 'j'): 19,
 ('c', 'i'): 21,
 ('c', 'o'): 23,
 ('c', 'p'): 25,
 ('e', 'c'): 4,
 ('e', 'i'): 25,
 ('e', 'f'): 6,
 ('e', 'b'): 6,
 ('e', 'g'): 8,
 ('e', 'a'): 8,
 ('e', 'h'): 10,
 ('e', 'd'): 10,
 ('e', 'k'): 13,
 ('e', 'n'): 17,
 ('e', 'm'): 19,
 ('e', 'l'): 19,
 ('e', 'j'): 19,
 ('e', 'o'): 23,
 ('e', 'p'): 25,
 ('b', 'f'): 4,
 ('b', 'g'): 6,
 ('b', 'a'): 6,
 ('b', 'e'): 6,
 ('b', 'i'): 31,
 ('b', 'c'): 6,
 ('b', 'h'): 8,
 ('b', 'd'): 8,
 ('b', 'n'): 11,
 ('b', 'l'): 17,
 ('b', 'j'): 17,
 ('b', 'k'): 19,
 ('b', 'o'): 21,
 ('b', 'm'): 21,
 ('b', 'p'): 23,
 ('f', 'b'): 4,
 ('f', 'i'): 35,
 ('f', 'g'): 6,
 ('f', 'a'): 6,
 ('f', 'c'): 14,
 ('f', 'e'): 6,
 ('f', 'p'): 31,
 ('f', 'j'): 25,
 ('f', '

In [180]:
path[0]

{'@': 0,
 'g': 3,
 'a': 3,
 'f': 3,
 'b': 3,
 'h': 5,
 'd': 5,
 'e': 9,
 'c': 9,
 'n': 14,
 'k': 18,
 'l': 20,
 'j': 20,
 'm': 24,
 'o': 24,
 'p': 26,
 'i': 30}

In [246]:
maze = ["#########","#b.A.@.a#","#########"]

In [254]:
maze_dict = {}
for y, line in enumerate(maze):
    for x, val in enumerate(line):
        if val != "#":
            maze_dict[(x, y)] = val

In [256]:
G = nx.Graph()

for coords, val in maze_dict.items():
    if val == "#":
        continue
    G.add_node(coords)
    next_bits = (
            (coords[0]+1, coords[1]),
            (coords[0]-1, coords[1]),
            (coords[0], coords[1]+1),
            (coords[0], coords[1]-1)
        )
    for n in next_bits:
        if maze_dict.get(n):
            G.add_node(n)
            G.add_edge(coords, n)
    

In [257]:
maze_dict

{(1, 1): 'b',
 (2, 1): '.',
 (3, 1): 'A',
 (4, 1): '.',
 (5, 1): '@',
 (6, 1): '.',
 (7, 1): 'a'}

In [261]:
nx.nodes()

TypeError: nodes() missing 1 required positional argument: 'G'

In [267]:
for p in nx.all_simple_paths(G, (5,1), (1,1)):
    i = [maze_dict[x] for x in p]
    print(i)

['@', '.', 'A', '.', 'b']


In [268]:
def part1(lines):
    '''
    >>> part1("######### #b.A.@.a# #########".split())
    8
    >>> part1("######################## #f.D.E.e.C.b.A.@.a.B.c.# ######################.# #d.....................# ########################".split())
    86
    >>> part1("######################## #...............b.C.D.f# #.###################### #.....@.a.B.c.d.A.e.F.g# ########################".split())
    132
    >>> part1("################# #i.G..c...e..H.p# ########.######## #j.A..b...f..D.o# ########@######## #k.E..a...g..B.n# ########.######## #l.F..d...h..C.m# #################".split())
    136
    >>> part1("######################## #@..............ac.GI.b# ###d#e#f################ ###A#B#C################ ###g#h#i################ ########################".split())
    81
    '''
    all_keys = {(x, y): c
                for y, line in enumerate(lines) for x, c in enumerate(line)
                if c.islower()}
    starts = dict(all_keys)
    bots = []
    for bot, pos in enumerate((x, y) for y, line in enumerate(lines)
                              for x, c in enumerate(line) if c == '@'):
        starts[pos] = str(bot)
        bots.append(str(bot))
    all_paths = {}
    for pos, item in starts.items():
        queue, seen, item_paths = [(0, pos, frozenset())], {pos}, {}
        while queue:
            d, pos, doors = heapq.heappop(queue)
            x, y = pos
            c = lines[y][x]
            if c != item and c.islower():
                item_paths[c] = (d, doors)
                continue
            elif c.isupper():
                doors = doors | {c.lower()}
            for x, y in ((x - 1, y), (x, y - 1), (x, y + 1), (x + 1, y)):
                if (x, y) in seen or not (0 <= y < len(lines)):
                    continue
                line = lines[y]
                if not (0 <= x < len(line)):
                    continue
                c = line[x]
                if c not in '.@' and not c.isalpha():
                    continue
                heapq.heappush(queue, (d + 1, (x, y), doors))
                seen.add((x, y))
        all_paths[item] = item_paths
    queue, seen = [(0, tuple(bots), frozenset())], set()
    while queue:
        d, items, keys = heapq.heappop(queue)
        if (items, keys) in seen:
            continue
        if len(keys) == len(all_keys):
            return d
        seen.add((items, keys))
        for i, item in enumerate(items):
            for item2, value in all_paths[item].items():
                d2, doors = value
                if doors - keys:
                    continue
                items2 = list(items)
                items2[i] = item2
                heapq.heappush(queue, (d + d2, tuple(items2), keys | {item2}))

In [270]:

from collections import defaultdict
import fileinput
import heapq


part1(inputs)

4954

In [276]:
lines = ["#########","#b.A.@.a#","#########"]

In [277]:
all_keys = {(x, y): c
            for y, line in enumerate(lines) for x, c in enumerate(line)
            if c.islower()}
starts = dict(all_keys)

In [278]:
all_keys

{(1, 1): 'b', (7, 1): 'a'}

In [279]:
starts

{(1, 1): 'b', (7, 1): 'a'}

In [275]:
maze_dict = {}
key_dict = {}
for y, line in enumerate(maze):
    for x, val in enumerate(line):
        maze_dict[(x, y)] = val
        if val.islower():
            key_dict[(x,y)]=val